# Build Dataset
In this notebook, we create a dataset containing all fables

## Modules Installation

In [16]:
! pip install roman
! pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Modules Import

In [17]:
import json
import roman

separator = 75 * '-'

## Create tiny-lafontaine

### Download dataset

I had the pleasure to find the complete work of La Fontaine gathered on [this page](https://www.bacdefrancais.net/les-fables-de-la-fontaine-texte-integral.php). It seems that the fables have been scanned and converted to text using an OCR method, so there are some typos and character mistakes here and there, but most of what I read while building the json file seemed correct.

I copied it all in a text file and manually did the following edits, to speed up dataset creation:  

- Added title, author and source url on the first 3 lines
  
- All fables have an id number written in roman numerals. The header is either `FABLE ID` or just the `ID`. I made sure that they all use the `FABLE [ID]` format.

- In the 7th book, there are two fables combined in one (LE HERON, LA FILLE). As they are two different stories and to avoid confusing the model, I separated them in two, with ids `FABLE IV-I` and `FABLE IV-I`.
  
- Sometimes the fable's title is written on multiple lines. I made sure it's all on one line (the 2nd line, right below the FABLE ID). This will simplify getting the fable's title.

- Sometimes the source is specified in square brackets, like `[Esope]` (La Fontaine adapted countless of Esope's stories. In fact, some say he never invented a single story himself!). I deleted it as it may confuse the LLM.
  
- Similar, sometimes the person he wrote the fable for is specified, like `A MADEMOISELLE DE SÉVIGNÉ`. I deleted it.

- `LIVRE SEPTIEME` starts with a long note and a message addressed to Mme de Montespan. I deleted it.

- `LIVRE ONZIEME` ends with an Epilogue. I deleted it.

- `LIVRE DOUZIEME` contains notes addressed to different beneficiaries. I deleted them.

- I fixed some typos I randomly spotted here and there, but that would take too long to go over all of them.

The final file structure looks like this:

```python
Title
Author
URL

LIVRE PREMIER
FABLE I
TITLE
...
FABLE II
TITLE
...

LIVRE SECOND
FABLE I
TITLE
...
FABLE II
TITLE
...

...etc...
```

### Load dataset
Load all fables from text file and parse all fables into a json file

### Parse dataset content in a unique json file

We want to parse our dataset according to the following json structure, easy to read and extract informaton from:

```json
{
    "title": "Les Fables (Texte intégral)",
    "author": "Jean De La Fontaine",
    "url": "http://...",
    "books": [
        "LIVRE PREMIER": [
            "FABLE I": "TITLE\n...",
            "FABLE II": "...",
            "FABLE III": "...",
        ],
        "LIVRE SECOND": [
            "FABLE I": "...",
            "FABLE II": "...",
            "FABLE III": "..."
        ]
    ]
}
```

In [18]:
dataset_file = "dataset/Jean de la Fontaine - Texte Integral.txt"

# load dataset file as a list, skip empty lines
with open(dataset_file, encoding='utf-8') as file:
    dataset = [line.strip() for line in file.readlines() if line.strip()]

# print first fable
for i in range(len(dataset)):
    if dataset[i].startswith("FABLE II"):
        break
    print(dataset[i])


Les Fables (Texte intégral)
Jean de la Fontaine
https://www.bacdefrancais.net/les-fables-de-la-fontaine-texte-integral.php
LIVRE PREMIER
FABLE I
LA CIGALE ET LA FOURMI
La Cigale, ayant chanté
Tout L'Été,
Se trouva fort dépourvue
Quand la Bise fut venue.
Pas un seul petit morceau
De mouche ou de vermisseau.
Elle alla crier famine
Chez la Fourmi sa voisine,
La priant de lui prêter
Quelque grain pour subsister
Jusqu'à la saison nouvelle.
« Je vous paierai, lui dit-elle,
Avant l'Août, foi d'animal,
Intérêt et principal. »
La Fourmi n'est pas prêteuse :
C'est là son moindre défaut.
« Que faisiez-vous au temps chaud ?
Dit-elle à cette emprunteuse.
- Nuit et jour à tout venant
Je chantais, ne vous déplaise.
- Vous chantiez ? j'en suis fort aise :
Eh bien ! dansez maintenant. »


In [19]:
# create tiny-lafontaine dataset
tiny_lafontaine = []
for i in range(len(dataset))[3:]:

    # skip fable and book titles
    if dataset[i].startswith("LIVRE") or dataset[i].startswith("FABLE"):
        continue
    
    tiny_lafontaine.append(dataset[i])

# write tiny-lafontaine dataset to file
tiny_lafontaine_file = "dataset/tiny-lafontaine.txt"
with open(tiny_lafontaine_file, "w", encoding='utf-8') as file:
    for line in tiny_lafontaine:
        file.write(line + "\n")

In [20]:
# create empty dictionary to store parsed dataset
dt_dic = {}

# add title, author and url on the first 3 lines
dt_dic['title'] = dataset[0]
dt_dic['author'] = dataset[1]
dt_dic['url'] = dataset[2]
dt_dic['books'] = {}

# iterate through dataset list and parse fables into a dictionary
book_count = 0
fable_count = 0

try:
    for line in dataset[3:]:
        if line.startswith("LIVRE"):
            current_book = line
            dt_dic['books'][current_book] = {}
            book_count += 1
        elif line.startswith("FABLE"):
            current_fable = line
            dt_dic['books'][current_book][current_fable] = ""
            fable_count += 1
        else:
            dt_dic['books'][current_book][current_fable] += line + "\n"
        
        print(f"Loading... Books: {book_count}, Fables: {fable_count}", end="\r", flush=True)

except Exception as e:
    print(e)
    

# save json file
with open('dataset/tiny-lafontaine.json', 'w', encoding='utf-8') as file:
    json.dump(dt_dic, file, indent=4)

We end up with a json file containing 12 books and 242 fables. 

The exact number of fables La Fontaine wrote varies between 240 and 249, according to different sources. Some count personal messages and dedications as fables too, while I got rid of them for simplicity. 

So we should have most of his work in our dataset now.

### Read data from JSON file

Let's print all fable titles from our dataset to see if everything works as expected.

In [21]:
# load json file
with open('dataset/tiny-lafontaine.json', 'r', encoding='utf-8') as file:
    dt = json.load(file)

# print all fable id and title (first two lines of each fable)
book_count = 0
fable_count = 0
for book in dt['books']:
    book_count += 1
    print(separator)
    print(book)
    print(separator)
    for fable_id, fable in dt['books'][book].items():
        query = fable.split("\n")[0]
        print(f"{fable_id}: {query}")
        fable_count += 1
print(separator)

print(f"Books: {book_count}, Fables: {fable_count}")

---------------------------------------------------------------------------
LIVRE PREMIER
---------------------------------------------------------------------------
FABLE I: LA CIGALE ET LA FOURMI
FABLE II: LE CORBEAU ET LE RENARD
FABLE III: LA GRENOUILLE QUI SE VEUT FAIRE AUSSI GROSSE QUE LE BOEUF
FABLE IV: LES DEUX MULETS
FABLE V: LE LOUP ET LE CHIEN
FABLE VI: LA GÉNISSE, LA CHEVRE ET LA BREBIS, EN SOCIÉTÉ AVEC LE LION
FABLE VII: LA BESACE
FABLE VIII: L'HIRONDELLE ET LES PETITS OISEAUX
FABLE IX: LE RAT DE VILLE ET LE RAT DES CHAMPS
FABLE X: LE LOUP ET L'AGNEAU
FABLE XI: L'HOMME ET SON IMAGE
FABLE XII: LE DRAGON À PLUSIEURS TETES, ET LE DRAGON À PLUSIEURS QUEUES
FABLE XIII: LES VOLEURS ET L'ANE
FABLE XIV: SIMONIDE PRÉSERVÉ PAR LES DIEUX
FABLE XV: LA MORT ET LE MALHEUREUX
FABLE XVI: LA MORT ET LE BÛCHERON
FABLE XVII: L'HOMME ENTRE DEUX AGES, ET SES DEUX MAITRESSES
FABLE XVIII: LE RENARD ET LA CIGOGNE
FABLE XIX: L'ENFANT ET LE MAITRE D'ÉCOLE
FABLE XX: LE COQ ET LA PERLE
FABLE XXI: LES 

### Helpers
Let's define some helper methods to easily retrieve contents

In [22]:
def get_fable(query: str, match_case=False) -> list:
    """
    Return a list of fables matching a title or words.

    Args:
        query (str): Title or words to search for.
        match_case (bool): If True, title must match exactly (except upper/lower case).
    """

    fables = []

    for book in dt['books']:
        for _, fable_text in dt['books'][book].items():
            fable_title = fable_text.split("\n")[0]
            if match_case:
                # title must be exactly the same, apart from upper/lower case
                if query.lower() == fable_title.lower():
                    fables.append(fable_text)
                    break
            else:
                # title must be part of the fable title
                if query.lower() in fable_title.lower():
                    fables.append(fable_text)
    
    return fables

fable_list = get_fable(query="le corbeau et le renard", match_case=False)

print(f"Found {len(fable_list)} fables.")

for fable in fable_list:
    print(separator)
    print(fable)

Found 1 fables.
---------------------------------------------------------------------------
LE CORBEAU ET LE RENARD
Maître Corbeau, sur un arbre perché,
Tenait en son bec un fromage.
Maître Renard, par l'odeur alléché,
Lui tint à peu près ce langage :
« Et bonjour, Monsieur du Corbeau.
Que vous êtes joli ! que vous me semblez beau !
Sans mentir, si votre ramage
Se rapporte à votre plumage,
Vous êtes le Phénix des hôtes de ces Bois. »
A ces mots le corbeau ne se sent pas de joie :
Et pour montrer sa belle voix,
Il ouvre un large bec, laisse tomber sa proie.
Le Renard s'en saisit, et dit : « Mon bon Monsieur,
Apprenez que tout flatteur
Vit aux dépens de celui qui j'écoute.
cette leçon vaut bien un fromage sans doute. »
Le corbeau honteux et confus
Jura, mais un peu tard, qu'on ne l'y prendrait plus.



In [23]:
def get_fable_by_id(book_id: int, fable_id: int) -> list:
    """
    Return a fable from a given book and id
    """
    book_ids = [
        'LIVRE PREMIER', 'LIVRE DEUXIEME', 'LIVRE TROISIEME', 'LIVRE QUATRIEME',
        'LIVRE CINQUIEME', 'LIVRE SIXIEME', 'LIVRE SEPTIEME', 'LIVRE HUITIEME',
        'LIVRE NEUVIEME', 'LIVRE DIXIEME', 'LIVRE ONZIEME', 'LIVRE DOUZIEME'
        ]
    
    fables = []
    
    try:
        if not 1 <= book_id <= 12:
            raise Exception("Invalid book id.")
        
        # convert ids to strings as formatted in json file
        book_id = book_ids[book_id - 1]
        fable_id = "FABLE " + roman.toRoman(fable_id)
        
        for id, text in dt['books'][book_id].items():
            if id == fable_id or id.startswith(fable_id + '-'):
                fables.append(f"{book_id} - {id}")
                fables.append(text)
        
        if not fables:
            raise Exception("Invalid fable id.")
        
        return fables
    
    except Exception as e:
        print(f"Could not find fable: {e}")
        return None

fable_list = get_fable_by_id(book_id=1, fable_id=3)
for fable in fable_list:
    print(fable)

LIVRE PREMIER - FABLE III
LA GRENOUILLE QUI SE VEUT FAIRE AUSSI GROSSE QUE LE BOEUF
Une Grenouille vit un Boeuf
Qui lui sembla de belle taille.
Elle qui n'était pas grosse en tout comme un oeuf,
Envieuse s'étend, et s'enfle, et se travaille
Pour égaler l'animal en grosseur,
Disant : « Regardez bien, ma sœur,
Est-ce assez ? dites-moi : n'y suis-je point encore ?
- Nenni. - M'y voici donc ? - Point du tout. - M'y voilà ?
- Vous n'en approchez point. » La chétive pécore
S'enfla si bien qu'elle creva.
Le monde est plein de gens qui ne sont pas plus sages :
Tout Bourgeois veut bâtir comme les grands Seigneurs,
Tout petit Prince a des Ambassadeurs,
Tout Marquis veut avoir des Pages.

